Milestone2


In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

#kagglehub.login()


In [2]:
#!kaggle datasets download -d tonygordonjr/football-match-statistics-and-more
#!unzip /content/football-match-statistics-and-more.zip -d ./

In [3]:
teams_clean_col = ['fixture_id', 'team_id', 'team_name', 'fouls','yellow_cards', 'red_cards']
stats_clean_col = ['fixture_id', 'fixture_date', 'fixture_referee', 'league_id', 'league_name', 'teams_home_id', 'teams_home_name', 'teams_away_id', 'teams_away_name']
lineup_clean_col = ['fixture_id', 'fixture_timestamp', 'fixture_referee', 'league_id', 'league_name', 'teams_home_id', 'teams_home_name', 'teams_away_id', 'teams_away_name','player_id', 'player_name', 'player_role']
players_clean_col = ['fixture_id', 'team_id', 'team_name', 'player_id', 'player_name', 'fouls_committed', 'yellow_cards', 'red_cards']

fix_teams_clean = pd.read_csv("fixture_stats_teams_clean.csv",usecols=teams_clean_col)
fix_clean = pd.read_csv("fixture_stats_clean.csv",usecols=stats_clean_col)
fix_lineup_clean = pd.read_csv("fixture_lineup_clean.csv",usecols=lineup_clean_col)
fix_players_clean = pd.read_csv("fixture_stats_players_clean.csv",usecols=players_clean_col)

In [5]:
import pandas as pd

fix_lineup_clean["fixture_timestamp"] = pd.to_datetime(fix_lineup_clean["fixture_timestamp"], unit="s")

start_date = pd.Timestamp("2022-01-01")

fix_lineup_clean = fix_lineup_clean[fix_lineup_clean["fixture_timestamp"] >= start_date]

print(fix_lineup_clean.head())



     fixture_id  player_id  player_name  player_role fixture_referee  \
345     1151013    50793.0    A. Sejdić   Substitute        C. Penso   
350     1151013    51075.0    J. Willis  Starting XI        C. Penso   
988     1151044    51017.0   T. Bunbury  Starting XI       L. Szpala   
989     1151044    18884.0  S. Surridge  Starting XI       L. Szpala   
990     1151028    20345.0  D. Yearwood  Starting XI        V. Rivas   

      fixture_timestamp  league_id          league_name  teams_home_id  \
345 2024-06-19 23:30:00        253  Major League Soccer           1601   
350 2024-06-19 23:30:00        253  Major League Soccer           1601   
988 2024-06-30 00:30:00        253  Major League Soccer           9569   
989 2024-06-30 00:30:00        253  Major League Soccer           9569   
990 2024-06-23 00:30:00        253  Major League Soccer           9569   

    teams_home_name  teams_away_id   teams_away_name  
345      Toronto FC           9569      Nashville SC  
350      Tor

In [6]:
count1 = fix_lineup_clean.shape[0]
count3 = fix_clean.shape[0]
top_leagues = ["La Liga", "Serie A", "Bundesliga", "Premier League"]
fix_lineup_clean = fix_lineup_clean[fix_lineup_clean["league_name"].isin(top_leagues)]
fix_clean = fix_clean[fix_clean["league_name"].isin(top_leagues)]
count2 = fix_lineup_clean.shape[0]
count4 = fix_clean.shape[0]
print(f"Number of rows before filtering: {count1}")
print(f"Number of rows after filtering: {count2}")
print(f"Number of rows before filtering: {count3}")
print(f"Number of rows after filtering: {count4}")

Number of rows before filtering: 459651
Number of rows after filtering: 179630
Number of rows before filtering: 48877
Number of rows after filtering: 19755


In [7]:
count1 = fix_players_clean.shape[0]
player_ids_to_keep = fix_lineup_clean["player_id"].unique()
fix_players_clean = fix_players_clean[fix_players_clean["player_id"].isin(player_ids_to_keep)]
count2 = fix_players_clean.shape[0]
print(f"Number of rows before filtering: {count1}")
print(f"Number of rows after filtering: {count2}")

Number of rows before filtering: 1113417
Number of rows after filtering: 514253


In [8]:
count1 = fix_teams_clean.shape[0]
team_ids_to_keep = fix_players_clean["team_id"].unique()
fix_teams_clean = fix_teams_clean[fix_teams_clean["team_id"].isin(team_ids_to_keep)]
count2 = fix_teams_clean.shape[0]
print(f"Number of rows before filtering: {count1}")
print(f"Number of rows after filtering: {count2}")

Number of rows before filtering: 65220
Number of rows after filtering: 61708


In [9]:
merged = fix_players_clean.merge(fix_lineup_clean, on="fixture_id", how="inner")
merged = merged.merge(fix_teams_clean, on="fixture_id", how="inner")
merged = merged.merge(fix_clean, on="fixture_id", how="inner")